In [ ]:
import json

In [ ]:
pokemon_card = {
    "name": "Pikachu",
    "type": "Electric",
    "HP": 100
}

schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "type": {"type": "string"},
        "HP": {"type": "integer"}
    },
    "required": ["name", "type", "HP"]
}

example = """{
    "name": "Pikachu",
    "type": "Electric",
    "HP": "100"
}
"""


schema_string = """
{
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "type": {"type": "string"},
        "HP": {"type": "integer"}
    },
    "required": ["name", "type", "HP"]
}
"""

In [ ]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(
  api_key=userdata.get("TOGETHER_API_KEY"),
  base_url="https://api.together.xyz/v1",
)

completion = client.chat.completions.create(
  model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
  messages=[
    {"role": "user", "content": "Generate a Pokemon card"}
  ],
  response_format={ "type": "json_object", "schema": schema }
)

print(completion.choices[0].message.content)


{"name": "Giratina", "type": "Ghost/Dragon", "HP": 130}


In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "calculator",
      "description": "Useful for when you need to answer questions about math. Use SymPy expressions",
    "parameters": {
      "type": "object",
      "properties": {
        "expression": {"type": "string", "description": "The expression to evaluate"}
      },
      "required": ["expression"]
    }
    }
  },
   {
    "type": "function",
    "function": {
      "name": "check_weather",
      "description": "Useful for when you need to check the weather",
    "parameters": {
      "type": "object",
      "properties": {
        "city_name": {"type": "string", "description": "Name of city NOT the country"}
      },
      "required": ["expression"]
    }
    }
  }
]

completion = client.chat.completions.create(
  model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
  messages=[
    {"role": "user", "content": "What's the weather in France today?"}
  ],
  tools=tools
)

In [ ]:
print(completion.choices[0].message.tool_calls[0].function)

Function(arguments='{"city_name":"Paris"}', name='check_weather')


In [ ]:
print(completion.choices[0].message.tool_calls[0].function.arguments)

{"city_name":"Paris"}


In [ ]:
print(completion.choices[0].message.tool_calls[0].function.name)

check_weather


In [ ]:
import requests

def get_weather_by_city(city):
    api_key = userdata.get('OPENWEATHER_API_KEY')  # Replace with your OpenWeather API key
    url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        data = response.json()
        print(data)  # You can return or process the data as needed
        return data
    except requests.exceptions.HTTPError as err:
        print(f'HTTP error occurred: {err}')
    except Exception as e:
        print(f'An error occurred: {e}')

# Example usage
get_weather_by_city('Paris')

{'coord': {'lon': 2.3488, 'lat': 48.8534}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 3.84, 'feels_like': 3.84, 'temp_min': 3.84, 'temp_max': 3.84, 'pressure': 1034, 'humidity': 75, 'sea_level': 1034, 'grnd_level': 1024}, 'visibility': 10000, 'wind': {'speed': 1.32, 'deg': 50, 'gust': 1.32}, 'clouds': {'all': 3}, 'dt': 1737137440, 'sys': {'country': 'FR', 'sunrise': 1737099425, 'sunset': 1737131036}, 'timezone': 3600, 'id': 2988507, 'name': 'Paris', 'cod': 200}


{'coord': {'lon': 2.3488, 'lat': 48.8534},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 3.84,
  'feels_like': 3.84,
  'temp_min': 3.84,
  'temp_max': 3.84,
  'pressure': 1034,
  'humidity': 75,
  'sea_level': 1034,
  'grnd_level': 1024},
 'visibility': 10000,
 'wind': {'speed': 1.32, 'deg': 50, 'gust': 1.32},
 'clouds': {'all': 3},
 'dt': 1737137440,
 'sys': {'country': 'FR', 'sunrise': 1737099425, 'sunset': 1737131036},
 'timezone': 3600,
 'id': 2988507,
 'name': 'Paris',
 'cod': 200}

In [ ]:
chat_history=[
    {
        "role": "user",
        "content": "What's the weather in France today?"
    }
  ]

while True:

  completion = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    messages=chat_history,
    tools=tools
  )

  tool_calls = completion.choices[0].message.tool_calls


  if tool_calls:
    arguments = tool_calls[0].function.arguments
    print(arguments)
    name = tool_calls[0].function.name
    print(name)

    chat_history.append(completion.choices[0].message)

    if name == "check_weather":
      city = json.loads(arguments)["city_name"]
      weather_data = get_weather_by_city(city)
      chat_history.append({
          "role": "function",
          "name": name,
          "content": json.dumps(weather_data)
      })
  else:
    print(completion.choices[0].message.content)
    break

{"city_name":"Paris"}
check_weather
{'coord': {'lon': 2.3488, 'lat': 48.8534}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 3.84, 'feels_like': 3.84, 'temp_min': 3.84, 'temp_max': 3.84, 'pressure': 1034, 'humidity': 75, 'sea_level': 1034, 'grnd_level': 1024}, 'visibility': 10000, 'wind': {'speed': 1.32, 'deg': 50, 'gust': 1.32}, 'clouds': {'all': 3}, 'dt': 1737137401, 'sys': {'country': 'FR', 'sunrise': 1737099425, 'sunset': 1737131036}, 'timezone': 3600, 'id': 2988507, 'name': 'Paris', 'cod': 200}
The weather in Paris, France today is clear with a temperature of 3.84 degrees Celsius and a humidity of 75%. The wind speed is 1.32 meters per second, and the visibility is 10 kilometers.
